# How to develop a C++ project using OpenVINO and run it on DevCloud

Here, you'll learn how to develop a image classification C++ application using OpenVINO, and how to run it on the DevCloud.
- Creating OpenVINO C++ project and build it (CMake)
- Running the built binary on the DevCloud development server
- Running the built binary on an edge computing node

----
## 1. Create and build C++ application and run it on the DevCloud development server

### 1.1. Prepareing a DL model
Let's start with downloading a DL model for this exercise. In this exercise, we'll use an image classification sample program, so you need to download an appropriate model for that purpose. This time, we'll download a `squeezenet1.1` model.  
Use an OpenVINO support tool `Model downloader` to download the model, and convert the downloaded model into OpenVINO IR model using `Model converter`. After you run `Model converter`, you'll see 5 `[ SUCCESS ]` at the end of the log output and the model conversion is successfully finished. The converted IR model files can be found in `./public/squezenet1.1/FP16/` directory.  

**Memo:** The `Model optimizer` in OpenVINO is the primary tool to convert a trained deep learning models generated by generic DL frameworks such as TensorFlow, ONNX, and Caffe. The `Model converter` is a front-end tool for the `Model optimizer` and it will simplify model conversion. The `Model converter` is a handy tool but it works only for the DL models downloaded with the `Model downloader`

In [ ]:
!python3 $INTEL_OPENVINO_DIR/deployment_tools/tools/model_downloader/downloader.py --name squeezenet1.1
!python3 $INTEL_OPENVINO_DIR/deployment_tools/tools/model_downloader/converter.py  --name squeezenet1.1 --precisions FP16

### 1.2. Preparing an image file and class label text file for inferencing
- Input image file
 - Copy an image file from OpenVINO demo directory (`car.png`)
- Class label text file
 - The `squeezenet1.1` model is trained with the ImageNet data-set which has 1000 classes
 - It is almost impossible to understand the result if program just output the class number, we'll use the class label text data (`synset_words.txt`) and output the class name in the program

In [ ]:
# Copy an image file
!cp $INTEL_OPENVINO_DIR/deployment_tools/demo/car.png .
# Download a class label text file
!curl -O https://raw.githubusercontent.com/HoldenCaulfieldRye/caffe/master/data/ilsvrc12/synset_words.txt

from IPython.display import Image
Image('car.png')

### 1.3. Preparing a C++ source code
For the sake of simplicity, we'll use `%%writefile` magic command to generate a C++ source code file here. You can use other common and generic way such as using a text editor, or uploading your source code to the server.  
The program is using OpenVINO to classify what's in the picture using deep leaning algorithm. We don't explain the source code or OpenVINO API here but the code is only 50+ lines and very simple.

In [ ]:
%%writefile main.cpp
#include <iostream>
#include <fstream>
#include <vector>

#include <opencv2/opencv.hpp>
#include <inference_engine.hpp>

namespace ie = InferenceEngine;

int main(int argc, char *argv[]) {

  std::ifstream label_file("synset_words.txt");  
  std::string str;
  std::vector<std::string> labels;
  while(getline(label_file, str)) labels.push_back(str);
  label_file.close();

  // Creating an Inference Engine core object
  ie::Core ie;

  // Loading a DL model to memory
  ie::CNNNetwork network = ie.ReadNetwork("public/squeezenet1.1/FP16/squeezenet1.1.xml",
                                          "public/squeezenet1.1/FP16/squeezenet1.1.bin");
  // Setting up the input blob
  std::shared_ptr<ie::InputInfo> input_info = 
                           network.getInputsInfo().begin()->second;
  std::string input_name = network.getInputsInfo().begin()->first;
  input_info->getPreProcess().setResizeAlgorithm(ie::RESIZE_BILINEAR);
  input_info->setLayout(ie::Layout::NHWC);
  input_info->setPrecision(ie::Precision::U8);

  // Setting up the output blob
  ie::DataPtr output_info = network.getOutputsInfo().begin()->second;
  std::string output_name = network.getOutputsInfo().begin()->first;
  output_info->setPrecision(ie::Precision::FP32);

  // Set the DL model to an Inference Engine object
  ie::ExecutableNetwork executable_network = ie.LoadNetwork(network, "CPU");
  ie::InferRequest infer_request = executable_network.CreateInferRequest();

// main inferencing loop - start -------

  cv::Mat image = cv::imread("car.png");   // Loading an image

  ie::TensorDesc tDesc(ie::Precision::U8, 
      {1, 3, static_cast<long unsigned int>(image.rows), 
             static_cast<long unsigned int>(image.cols) }, ie::Layout::NHWC);
  infer_request.SetBlob(input_name, ie::make_shared_blob<uint8_t>(tDesc, image.data));

  infer_request.Infer();      // Do inferencing

  // Displaying result
  float* output = infer_request.GetBlob(output_name)->buffer();
  std::cout << "\nresults\n------------------" << std::endl;
  std::vector<int> idx;
  for(int i=0; i<1000; i++) idx.push_back(i);
  std::sort(idx.begin(), idx.end(), [output](const int& left, const int& right) { return output[left]>output[right]; } );
  for (size_t id = 0; id < 5; ++id)  std::cout << id <<  " : " << idx[id] << " : " << 
    std::fixed<< std::setprecision(2) << output[idx[id]]*100 << "% " << labels[idx[id]] << std::endl;

// main inferencing loop - end ------
}

### 1.4. Preparing `CMakeLists.txt` for building the source
Create a `cmake` configuration file (`CMakeLists.txt`) using `%%writefile` magic command.
You can reuse this configuration file for your own project by just modifying `set(TARGET_NAME` and `add_executable(` part appropriately. You may need to add libraries to `target_link_libraries()` or add `find_package()` if you use extra libraries in your project.

In [ ]:
%%writefile CMakeLists.txt
# Sample CMakeLists.txt file for an OpenVINO Inference Engine project
cmake_minimum_required (VERSION 2.8.1)

set(TARGET_NAME simple_cnn)     # name of executable file
set(CMAKE_BUILD_TYPE "Release")

set(CMAKE_CXX_STANDARD 11)
set(CMAKE_CXX_STANDARD_REQUIRED ON)
set(CMAKE_CXX_FLAGS "${CMAKE_CXX_FLAGS} -std=c++11 -fPIE")

find_package(InferenceEngine 1.1 REQUIRED)
find_package(OpenCV REQUIRED)
add_definitions(-DUSE_OPENCV)

include_directories( ${InferenceEngine_INCLUDE_DIRS} ${OpenCV_INCLUDE_DIRS} )
link_directories( )

add_executable( ${TARGET_NAME} main.cpp )    # list of source file(s)
set_target_properties(${TARGET_NAME} PROPERTIES "CMAKE_CXX_FLAGS" "${CMAKE_CXX_FLAGS}")
target_link_libraries(${TARGET_NAME} ${InferenceEngine_LIBRARIES} ${OpenCV_LIBS} )


### 1.5. Building the application
Create a `build` directory and build the C++ source code.  
The `!` command will invoke a child-shell, run the command and destroy the child-shell every time you run a command. So the change you made for the shell will be discarded and won't take effect.  
For example, the `cd` command will change the working directory but it is effective until the end of the line.  
So, we need to put everything in a line using `&&`.

The built binary can be found as `./build/simple_cnn`.

In [ ]:
!mkdir -p build && cd build && cmake .. && make && ls -l

### 1.6. Running the built binary on the DevCloud development server
Run the build binary **on the DevCloud development server**.  
This program doesn't take any option parameters. The input image file name, model file name, and class label file name are hard coded in the source code.  

Run the binary and check if the output is correct and expected.

In [ ]:
!build/simple_cnn

----
## 2. Submitting a job and run a C++ program on a DevCloud edge computing node

In the previous section, we run the C++ program on the development server.  
Here, we'll submit a job and run the same program on an edge computing node on the DevCloud.

### 2.1. Creating a job script file
The working directory of your job on an edge computing node is the user home directory (`~`, , `$HOME`, `/home/<userid>`), regardless of your current working directory on the host server. 
Please change the working directory appropriately and make sure that your job is working on the desired directory on the edge computing node.  
**The same storage is visible from the edge computing node**. You don't need to transfer related files such as input image file to the edge computing node.

In [ ]:
%%writefile job.sh
cd ~/devcloud-workshop-en
build/simple_cnn


### 2.2. Submitting and running a job on an edge computing node
We don't specify the specific edge computing node here, so, the job will be run on an edge computing node adhocery.

In [ ]:
# submit a job
job_id=!qsub job.sh

# generate log file name from job_id
job_num = job_id[0].split('.')[0]
log_file='job.sh.o'+job_num
err_file='job.sh.e'+job_num
print('job_id={}, log_file={}'.format(job_id, log_file))

import time
def waitForJobCompletion(jobNumber):
    print('Waiting for job completion...', end='')
    running=True
    while running:
        time.sleep(1)
        running=False
        status_list=!qstat         # Check job status
        for status in status_list:
            if jobNumber in status:    # if job_num is found in the status list, the job is still running
                running = True
        print(status.split()[4], end='')
    print('...Job {} completed'.format(job_num))    

# wait for the job to complete
waitForJobCompletion(job_num)

### 2.3. Checking the output log
Display the output log file (`job.sh.o<job#>`) and confirm whether the same result is output as when we run it on the development server or not.

In [ ]:
# The log_file is set in the previous cell
import os
os.environ['log_file']=log_file

!cat $log_file

----
Now, you have learnt how to create, build and run a C++ project on DevCloud.
Let's modify the sample program here or, try your own code on DevCloud.

End of the course

## < Appendix > [Automate evaluation work on DevCloud](./automated-testing.ipynb>automated-testing.ipynb)